In [4]:
import pandas as pd
import dash
from dash import dcc, html
import plotly.express as px

# Load the processed data
merged_df = pd.read_csv('data/processed/merged_shipments.csv')

# Calculate summary metrics
total_shipments = len(merged_df)
on_time = len(merged_df[merged_df['status'] == 'on-time'])
delayed = len(merged_df[merged_df['status'] == 'delayed'])
on_time_rate = round((on_time / total_shipments) * 100, 2)

# Group by vendor
vendor_perf = merged_df.groupby('name')['status'].value_counts(normalize=True).unstack().fillna(0)
vendor_perf['on_time_rate'] = vendor_perf.get('on-time', 0)

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Supply Chain Monitoring Dashboard"

# Layout
app.layout = html.Div([
    html.H1("Shipment Performance Dashboard", style={'textAlign': 'center'}),
    
    html.Div([
        html.Div([
            html.H3("Total Shipments"),
            html.P(f"{total_shipments}")
        ], className="metric"),

        html.Div([
            html.H3("On-Time Shipments"),
            html.P(f"{on_time}")
        ], className="metric"),

        html.Div([
            html.H3("Delayed Shipments"),
            html.P(f"{delayed}")
        ], className="metric"),

        html.Div([
            html.H3("On-Time Rate"),
            html.P(f"{on_time_rate}%")
        ], className="metric"),
    ], style={'display': 'flex', 'justifyContent': 'space-around'}),

    html.Br(),

    dcc.Graph(
        figure=px.bar(
            vendor_perf,
            x=vendor_perf.index,
            y='on_time_rate',
            title='On-Time Rate by Vendor',
            labels={'on_time_rate': 'On-Time Rate', 'name': 'Vendor'},
            text_auto='.2f'
        )
    )
])

if __name__ == '__main__':
    app.run(debug=True)
